Rating systems can help us in solving a number of problems:

1. Ordering all players by skills. 
2. Predicting the winner. 
3. Forming teams of similar strength.

We will try to discuss each problem.

There are different rating systems. We will consider three of them:
1. Elo (https://en.wikipedia.org/wiki/Elo_rating_system)
2. Glicko/Glicko 2 (http://www.glicko.net/glicko.html)
3. TrueSkill (https://www.microsoft.com/en-us/research/project/trueskill-ranking-system/)

For better understanding of TrueSkill model we higly recommend to read excelent description by Jeff Moser (http://www.moserware.com/2010/03/computing-your-skill.html) and related paper (http://www.moserware.com/assets/computing-your-skill/The%20Math%20Behind%20TrueSkill.pdf). This links also relevant to Elo ratings, since it considered as basic model.


We will use dataset from http://dotascience.com/. You can obtain data by link http://dotascience.com/data/dotahack_online_data.zip. At this class we will use only information about matches results (selected_team_matches.csv), rather than all data.

The main idea of rating systems is to find parameters in specific math models that gives better prediction of winer, or better prediction of scores obtained in one game.

For Elo rating we have a simple model that estimates expected score for player A from one game with player B as $$E_A = \frac 1 {1 + 10^{(R_B - R_A)/400}},$$ where $R_A$ and $R_B$ are current ratings of player A and player B respectively.

Obviously:
* If $R_A = R_B$ it means that estimated score is equal 0.5.
* If $R_A = R_B + 400$ it means that estimated score is equal 10/11, i.e. player A got ten times more score than player B. If the only outcomes are win or lose (no draws), player A wins ten times more frequently than player B i.e. the probability of wining is 10/11.

***Question:*** Can you calculate rating difference that provides player A an estimate of 75% of the total score?

You should get approxemately 190.85.

The model above provides you with a chances of win or estimation of scores with given players ratings, but does not give you any clue how to estimate this ratings.

The idea is very simple: if you score more than expected you get rating boost, if you score less than expected you lose some score points.

The exact formula for changes in Elo rating of player A is $$R_{A}^{\prime }=R_{A}+K(S_{A}-E_{A}),$$ where $S_A$ is real score of player A, $E_A$ - expected scores estimated by formula described above, an $K$ is a coefficient controlling the speed of rating changes. Usualy we use large $K$ for newcomers and smaller $K$ for players with a long history. You may check [wiki page](https://en.wikipedia.org/wiki/Elo_rating_system#Most_accurate_K-factor) for examples of different K schemas.

_Note_: if you use the same $K$ for each player you just redistribute rating points among players.


The last thing that we should discuss in connection with Elo rating is initialisation of rating for new players. 

Since the only factor that affect our predictions is the difference in ratings rather than their absolute value, we can arbitrary chouse any value for newbees. Usualy this is either an exact number e.g. 1000, or mean/median value of all players. In our model we will use 1000 points as a start rating.

In [ ]:
Elo rating system is very simple, so we can reimpliment it for an educational purpose. We will need a function that recalculate ratings with respect to the match outcome. It also will be usefull to have a fuction estimating chance of win, and since there are no draws in our it will be very straitforward.

In [1]:
def Elo_win_probability(player_rating, opponent_rating):
    delta_rating = player_rating - opponent_rating
    return 1/(1+10**(-delta_rating/400.))

Lets check if our function return expected results on few example cases:

In [2]:
Elo_win_probability(1000, 1000)

0.5

In [3]:
Elo_win_probability(1190.85, 1000)

0.7500016169640152

It provides us with expected results. Let's implement function that returns updated ratings. We will use the same $K=30$ for every game, but you may implement $K$ changes later.  

In [4]:
def Elo_update_rates(winer_rating, loser_rating, winer_K = 30, loser_K = 30):
    winer_estimated_score = Elo_win_probability(winer_rating, loser_rating)
    loser_estimated_score = 1 - winer_estimated_score # Estimated scores should sum up to 1    
    winer_new_rating = winer_rating + winer_K *(1 - winer_estimated_score)
    loser_new_rating = loser_rating + loser_K *(0 - loser_estimated_score)
    return (winer_new_rating, loser_new_rating)

In [5]:
Elo_update_rates(1000, 1000)

(1015.0, 985.0)

Now we have function that recalculate rating, so we can calculate ratings for each team in our dataset. We will consider each game separately, but you shoud later think how to deal with "best of 3" or "best of 5" series as with one event.

In [6]:
import pandas as pd
matches_info = pd.read_csv('selected_team_matches.csv').sort_values(['match_id'])

In [7]:
matches_info.head()

,date,tournament,radiant,dire,winner,match_id
0,2012-06-24,The Defense 2,WhA,EG,DIRE,22270148
1,2012-06-28,The Defense 2,Unknown,EG,DIRE,22959375
2,2012-06-29,StarSeries II Finals,EG,Empire,DIRE,23152391
3,2012-06-29,StarSeries II Finals,EG,NEXT.kz,RADIANT,23160256
4,2012-07-11,The Defense 2,EG,Unknown,DIRE,25449321


Now we need to create a dictionary with current rates. After that we iterate through all matches and recalculate ratings acording to each match outcome. 

In [8]:
Elo_ratings = {}
start_rating = 1000

In [9]:
for index, row in matches_info.iterrows():
    team_radiant = row['radiant']
    team_dire = row['dire']
    if team_radiant not in Elo_ratings:
        Elo_ratings[team_radiant] = start_rating
    if team_dire not in Elo_ratings:
        Elo_ratings[team_dire] = start_rating
    if  row['winner'] == "RADIANT":
        Elo_ratings[team_radiant], Elo_ratings[team_dire] = Elo_update_rates(Elo_ratings[team_radiant], Elo_ratings[team_dire])
    else:
        Elo_ratings[team_dire], Elo_ratings[team_radiant] = Elo_update_rates(Elo_ratings[team_dire], Elo_ratings[team_radiant])
        

Now we have Elo ratings of every team. Lets find the leaders and losers. 

In [10]:
team_sorted_by_Elo = sorted(Elo_ratings, key=Elo_ratings.get, reverse=True)

In [11]:
team_sorted_by_Elo[:5]

['EHOME', 'Alliance', 'OG', 'VP 2', 'EG']

In [12]:
[Elo_ratings[team] for team in team_sorted_by_Elo[:5]]

[1391.782235380362,
 1347.6644664248986,
 1311.7109234075542,
 1277.8484821634563,
 1276.8960645309862]

In [13]:
team_sorted_by_Elo[-5:]

['SiG.Tr', 'LvT', 'EHUG', 'DK', '4ASC']

In [14]:
[Elo_ratings[team] for team in team_sorted_by_Elo[-5:]]

[912.6701186183126,
 906.1960947912388,
 896.9281893130452,
 894.738858505895,
 888.0133892079333]

Now we can estimate winning chances of Virtus pro (with a tag 'VP 2') in the match against Natus Vincere (with a tag 'NaVi').

In [15]:
Elo_win_probability(Elo_ratings['VP 2'], Elo_ratings['NaVi'])

0.7911955410911068

Now you know the basics of rating systems. 

Elo rating system is good and quite useful but it has some issues. The main issue of Elo system is the lack of information about player's rating uncertainty. Consider the person with rating 1000. What does it mean? If the person played 1000 games it means that it is an average player, but if the person played only 2 games it means almost nothing. 
How can we deal with such uncertanty?
And can we deal with teams of players that can change from game to game?

The answers for questions above is TrueSkill. The TrueSkill is the system that allows you to compare "teams" that are not stable when it comes to players. For example if I play any sesional game that assigns me to a random team in each match how the result of game should affect on estimation of my perfomance? Another possible case is if we have teams with defined members, but often one or two of this members can not play and team plays with a guest member, usualy caled "legioner". Shoud we use information of such game to evaluate team rating or not?

TrueSkill considers following basic ideas:
1. Each player has his own rating defined by mean and variance.
2. In each game we assume that player perfomance in the game is a random value normally distributed with mean and variance from the player's rating.
3. Team perfomance in each game is a sum of performance of players in this game. 

All above lead to the notion of team perfomance in every game as some normaly distributed random value, so we can estimate winning probabilities. And since the model can be described in terms of random variables distributions we can use Bayes theorem to update player ratings with respect to outcome. Since the team rating is just sum of player ratings, we can ignore teams: if we nead team rating we just calculate it based on member list.

First of all lets use TrueSkill for estimates rating for the same data that we use previously. We will use package [**trueskill**](http://trueskill.org/).

Since we have no information about players we will consider each team as a team from one player, same player in each match of that team.

In [16]:
import trueskill
from trueskill import BETA
from trueskill.backends import cdf
from math import sqrt

The main process of ratin evaluation for TrueSkill is the same as for Elo rating. We just need to implement new function for update rating based on one match. And we also need to have ability to calculate chances to win. The estimation of win probabilities is based on intersect two normal distribution. 

Before using TrueSkill we should initialise it with parameters that fit to our game better. You may try to tune this parameters for better results, but now we use default values. The only thing that we change is the draw_probability, since the draw is impossible we set in to 0.

In [17]:
trueskill.setup(draw_probability=0)

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%)

In [18]:
def TrueSkill_win_probability(player_rating, opponent_rating):
    delta_mu = player_rating.mu - opponent_rating.mu
    denom = sqrt(2 * (BETA * BETA) + pow(player_rating.sigma, 2) + pow(opponent_rating.sigma, 2))
    return cdf(delta_mu / denom)

In [19]:
def TrueSkill_update_rates(winer_rating, loser_rating):
    return trueskill.rate_1vs1(winer_rating, loser_rating)

Now we can reimpliment Elo case main loop.

In [20]:
TS_ratings = {}

In [21]:
for index, row in matches_info.iterrows():
    team_radiant = row['radiant']
    team_dire = row['dire']
    if team_radiant not in TS_ratings:
        TS_ratings[team_radiant] = trueskill.Rating()
    if team_dire not in TS_ratings:
        TS_ratings[team_dire] = trueskill.Rating()
    if  row['winner'] == "RADIANT":
        TS_ratings[team_radiant], TS_ratings[team_dire] = TrueSkill_update_rates(TS_ratings[team_radiant], TS_ratings[team_dire])
    else:
        TS_ratings[team_dire], TS_ratings[team_radiant] = TrueSkill_update_rates(TS_ratings[team_dire], TS_ratings[team_radiant])
        

In [22]:
team_sorted_by_TS = sorted(TS_ratings, key=lambda x:(TS_ratings[x].mu), reverse=True)

In [23]:
team_sorted_by_TS[:5]

['Lv', 'iCCup', 'Super Strong', 'coL', 'EED']

In [24]:
[TS_ratings[team] for team in team_sorted_by_TS[:5]]

[trueskill.Rating(mu=36.183, sigma=4.976),
 trueskill.Rating(mu=35.056, sigma=5.101),
 trueskill.Rating(mu=34.588, sigma=5.159),
 trueskill.Rating(mu=34.479, sigma=3.812),
 trueskill.Rating(mu=33.755, sigma=5.381)]

The 5 team above is have nothing comon with Elos top 5. What does it mean?

One of the possible answers is the lack of data. The standard deviation of all this teams is very large, so it means that real team perfomance may be almost any. We know that for normaly distributed variable the 95% confidence interval can be evaluated as $[\mu-1.96*\sigma,\mu+1.96*\sigma]$, so we can use the lower bound as a conservative estimate. We almost sure that rating is al least $\mu-1.96*\sigma$. Let sort the teams according conservative estimates. 

In [25]:
team_sorted_by_TS_conservative = sorted(TS_ratings, key=lambda x:(TS_ratings[x].mu - 1.96*TS_ratings[x].sigma), reverse=True)

In [26]:
team_sorted_by_TS_conservative[:5]

['OG', 'EHOME', 'EG', 'VG', 'VP 2']

In [27]:
[TS_ratings[team] for team in team_sorted_by_TS_conservative[:5]]

[trueskill.Rating(mu=33.187, sigma=0.809),
 trueskill.Rating(mu=33.006, sigma=0.801),
 trueskill.Rating(mu=32.591, sigma=0.798),
 trueskill.Rating(mu=32.472, sigma=0.798),
 trueskill.Rating(mu=32.329, sigma=0.799)]

And now we get the similar teams in slightly different ordes.

As we already mentioned, TrueSkill also allow us to take the composition of teams into account. Lets try it.
We will use another Dota 2 dataset. Use file prise_matches.csv. It slightly differs from previous one, but it provides us with players information. 

In [28]:
matches_info_players = pd.read_csv('prize_matches.csv').sort_values(['start_time'])

In [29]:
players_rates = {}

In [30]:
def TrueSkill_win_probability_by_players(T1_ratings, T2_ratings):#You shoud use the list of players ratings
    Team1_mu = 0
    Team1_sigma = 0
    Team2_mu = 0
    Team2_sigma = 0
    
    for r in T1_ratings:
        Team1_mu += r.mu
        Team1_sigma += BETA * BETA + r.sigma * r.sigma
    
    for r in T2_ratings:
        Team2_mu += r.mu
        Team2_sigma += BETA * BETA + r.sigma * r.sigma

    delta_mu = Team1_mu - Team2_mu
    denom = sqrt(Team1_sigma + Team2_sigma)
    return cdf(delta_mu / denom)

In [31]:
for index, row in matches_info_players.iterrows():
    radiant_team_id = row['radiant_team_id']
    dire_team_id = row['dire_team_id']
    
    radiant_team_players_ids = [] 
    dire_team_players_ids = [] 
    
    for i in range(5):
        radiant_team_players_ids.append(row['r'+str(i+1)+'_account_id'])
        dire_team_players_ids.append(row['d'+str(i+1)+'_account_id'])
    
    radiant_team_players_ratings = []    

    for p_id in radiant_team_players_ids:
        if p_id not in players_rates:
            players_rates[p_id] = trueskill.Rating()
        radiant_team_players_ratings.append(players_rates[p_id])
    
    dire_team_players_ratings = []    

    for p_id in dire_team_players_ids:
        if p_id not in players_rates:
            players_rates[p_id] = trueskill.Rating()
        dire_team_players_ratings.append(players_rates[p_id])
    
    
    if row['radiant_win']:
        res = trueskill.rate([radiant_team_players_ratings, dire_team_players_ratings], ranks=[0, 1])
    else:
        res = trueskill.rate([radiant_team_players_ratings, dire_team_players_ratings], ranks=[1, 0])

    for i in range(5):
        players_rates[radiant_team_players_ids[i]] = res[0][i]
        players_rates[dire_team_players_ids[i]] = res[1][i]  

Now we can find the top players. 

In [32]:
players_sorted_by_TS = sorted(players_rates, key=lambda x:(players_rates[x].mu), reverse=True)

In [33]:
players_sorted_by_TS[:5]

[3832921, 85844766, 31778508, 81904922, 110880087]

In [34]:
[players_rates[pl] for pl in players_sorted_by_TS[:5]]

[trueskill.Rating(mu=42.075, sigma=4.853),
 trueskill.Rating(mu=41.728, sigma=4.684),
 trueskill.Rating(mu=41.728, sigma=4.684),
 trueskill.Rating(mu=41.613, sigma=4.704),
 trueskill.Rating(mu=37.602, sigma=4.100)]

In [35]:
players_sorted_by_TS_conservative = sorted(players_rates, key=lambda x:(players_rates[x].mu - 1.96 *players_rates[x].sigma), reverse=True)

In [36]:
players_sorted_by_TS_conservative[:5]

[111620041, 139876032, 86727555, 19672354, 38628747]

In [37]:
[players_rates[pl] for pl in players_sorted_by_TS_conservative[:5]]

[trueskill.Rating(mu=36.350, sigma=1.294),
 trueskill.Rating(mu=36.000, sigma=1.308),
 trueskill.Rating(mu=35.555, sigma=1.240),
 trueskill.Rating(mu=35.248, sigma=1.249),
 trueskill.Rating(mu=35.423, sigma=1.355)]

Can you list teams of each of this top players?

There are couple of task that can help you to check your understanding of code above.

**Task 1:** In every rating evaluation we store only current rating for each team or player. Improve the code to store rating history. Can you draw a graph that shows the rating history for some team e.g. Virtus pro, or any other one?

**Task 2:** We have not discussed assessing rating quality yet. Using predicted win probabilities, check which team has more chances to win before you update the ratings. 
Compare it to real results. Calculate accuracy (a share of correct predictions) for every 50 or 100 consecutive matches. Does accuracy tend to rise or to decrease with time?

**Task 3:** If the team does not play for a long time we shoud increase the uncertanty estimate of its rating. You have a date of each match. Before update rating of teams, calculate the number of days from the previous match, and add this number muliplied by some coefficient (e.g. 0.01) to sigma value of this team rating. Does such penalty affect model quality? 